# 🗄️ Module 1: Core Python & Data - Week 4 Lecture 19
**Date:** 12/09/2025  
**Documented By:** Muhammad Soban Shaukat

## 🔗 Integrating Python with SQL (Part 2): Writing to Databases

Welcome to Lecture 19! In our previous session, we learned how to read data from SQL databases using Python. Today, we'll learn how to write data to databases using Data Manipulation Language (DML) operations.

```python
# Setup: Import necessary libraries
import sqlite3
import pandas as pd
from datetime import datetime

print("✅ Libraries imported successfully!")
```

## 📋 Today's Agenda

1.  **📊 Recap: Python + SQL Part 1**
    *   Review of reading data from databases
    *   DB-API 2.0 flow: Connect → Cursor → Execute → Fetch → Close

2.  **📝 DML from Python: INSERT, UPDATE, DELETE**
    *   Writing data to databases programmatically
    *   Parameterized queries for safety

3.  **🎯 Dynamic Data Insertion**
    *   Collecting user input and inserting into database
    *   Validation and error handling

4.  **🛡️ Real-World Safeguards & Best Practices**
    *   Foreign key validation, transactions, soft deletion
    *   Security considerations

5.  **🧪 Hands-On Lab: Complete Blog Management System**
    *   Building a CLI application for blog management
    *   Bonus exercises for advanced practice

---

## 1. 📊 Recap: Python + SQL Part 1

### 🔄 DB-API 2.0 Flow Review

Let's quickly review what we learned in the previous lecture about reading data from databases:

```python
print("🔄 DB-API 2.0 FLOW REVIEW")
print("=" * 30)

dbapi_steps = [
    "1. CONNECT: sqlite3.connect('database.db')",
    "2. CURSOR: connection.cursor()",
    "3. EXECUTE: cursor.execute('SELECT ...')",
    "4. FETCH: cursor.fetchall() / fetchone() / fetchmany()",
    "5. CLOSE: connection.close()"
]

for step in dbapi_steps:
    print(f"   {step}")

print("\n💡 Today we focus on Step 3: EXECUTE for DML operations (INSERT, UPDATE, DELETE)")
```

### 📖 Reading vs Writing Operations

```python
print("📖 READING VS WRITING OPERATIONS")
print("=" * 40)

comparison = {
    "READING (SELECT)": "Retrieve data without changing database state",
    "WRITING (DML)": "Modify database state (INSERT, UPDATE, DELETE)",
    "Commit Required": "No commit needed for SELECT",
    "Commit Required ": "Must commit() after DML operations",
    "Risk Level": "Low risk - only reading data",
    "Risk Level ": "High risk - can modify or lose data"
}

print("🔍 SELECT OPERATIONS:")
print(f"   - {comparison['READING (SELECT)']}")
print(f"   - {comparison['Commit Required']}")
print(f"   - {comparison['Risk Level']}")

print("\n📝 DML OPERATIONS:")
print(f"   - {comparison['WRITING (DML)']}")
print(f"   - {comparison['Commit Required ']}")
print(f"   - {comparison['Risk Level ']}")
```

---

## 2. 📝 DML from Python: INSERT, UPDATE, DELETE

### 🏗️ Database Setup for Examples

Let's create a sample blog database for our DML operations:

```python
print("🏗️ SETTING UP BLOG DATABASE")
print("=" * 35)

# Create or connect to blog database
blog_conn = sqlite3.connect("blog.db")
blog_cur = blog_conn.cursor()

# Create Users table
blog_cur.execute("""
CREATE TABLE IF NOT EXISTS Users (
    UserID INTEGER PRIMARY KEY,
    Username TEXT UNIQUE NOT NULL,
    Email TEXT UNIQUE NOT NULL,
    DateJoined DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")

# Create Posts table with foreign key
blog_cur.execute("""
CREATE TABLE IF NOT EXISTS Posts (
    PostID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    Content TEXT,
    PublishedDate DATETIME DEFAULT CURRENT_TIMESTAMP,
    AuthorID INTEGER,
    IsActive INTEGER DEFAULT 1,
    FOREIGN KEY (AuthorID) REFERENCES Users(UserID)
)
""")

# Insert sample users if they don't exist
users = [
    (1, 'alice_dev', 'alice@example.com'),
    (2, 'bob_writer', 'bob@example.com'),
    (3, 'charlie_author', 'charlie@example.com')
]

blog_cur.executemany("""
INSERT OR IGNORE INTO Users (UserID, Username, Email)
VALUES (?, ?, ?)
""", users)

blog_conn.commit()
print("✅ Blog database setup complete")
print("   - Users table created with sample data")
print("   - Posts table created with foreign key constraint")
```

### 📥 INSERT Operation

```python
print("\n📥 INSERT OPERATION")
print("=" * 25)

# Example: Insert a new post
insert_query = """
INSERT INTO Posts (Title, Content, AuthorID)
VALUES (?, ?, ?)
"""

post_data = (
    "Getting Started with Python",
    "Python is a versatile programming language that's great for beginners...",
    1  # AuthorID for alice_dev
)

blog_cur.execute(insert_query, post_data)
blog_conn.commit()

print("✅ New post inserted successfully!")
print(f"   Title: {post_data[0]}")
print(f"   AuthorID: {post_data[2]}")
```

### 📝 UPDATE Operation

```python
print("\n📝 UPDATE OPERATION")
print("=" * 25)

# Example: Update a post's title
update_query = """
UPDATE Posts
SET Title = ?
WHERE PostID = ?
"""

update_data = ("Python Programming Fundamentals", 1)

blog_cur.execute(update_query, update_data)
blog_conn.commit()

print("✅ Post updated successfully!")
print(f"   New title: {update_data[0]}")
```

### 🗑️ DELETE Operation

```python
print("\n🗑️ DELETE OPERATION")
print("=" * 25)

# Example: Delete a post (with caution!)
delete_query = """
DELETE FROM Posts
WHERE PostID = ?
"""

# For safety, we won't actually execute this in the example
print("⚠️  DELETE operation would remove data permanently")
print("💡 Better approach: Use soft deletion (shown later)")
```

### 🔒 Parameterized Queries for Safety

```python
print("\n🔒 PARAMETERIZED QUERIES FOR SAFETY")
print("=" * 45)

# UNSAFE: String concatenation (VULNERABLE to SQL injection)
user_input = "Python'; DROP TABLE Posts; --"
unsafe_query = f"INSERT INTO Posts (Title) VALUES ('{user_input}')"

# SAFE: Parameterized queries (RECOMMENDED)
safe_query = "INSERT INTO Posts (Title) VALUES (?)"

print("❌ UNSAFE APPROACH:")
print(f"   {unsafe_query}")

print("\n✅ SAFE APPROACH:")
print(f"   {safe_query}")
print(f"   Parameters: ('{user_input}',)")

print("\n💡 Always use parameterized queries to prevent SQL injection attacks!")
```

---

## 3. 🎯 Dynamic Data Insertion

### 💻 Collecting User Input

Let's create a function that collects user input and inserts a new blog post:

```python
print("💻 DYNAMIC DATA INSERTION: USER INPUT")
print("=" * 50)

def add_post():
    """Collect user input and insert a new blog post"""
    try:
        with sqlite3.connect("blog.db") as conn:
            # Enable foreign key support
            conn.execute("PRAGMA foreign_keys = ON")
            cur = conn.cursor()
            
            # Collect user input
            print("\n📝 ADD NEW BLOG POST")
            print("-" * 30)
            
            title = input("Enter post title: ").strip()
            content = input("Enter post content: ").strip()
            author_id = input("Enter author ID: ").strip()
            
            # Validate input
            if not title:
                print("❌ Error: Title cannot be empty")
                return
                
            if not content:
                print("❌ Error: Content cannot be empty")
                return
                
            if not author_id.isdigit():
                print("❌ Error: Author ID must be a number")
                return
                
            author_id = int(author_id)
            
            # Verify author exists
            cur.execute("SELECT 1 FROM Users WHERE UserID = ?", (author_id,))
            if cur.fetchone() is None:
                print(f"❌ Error: Author with ID {author_id} does not exist")
                return
            
            # Insert the post
            cur.execute(
                "INSERT INTO Posts (Title, Content, AuthorID) VALUES (?, ?, ?)",
                (title, content, author_id)
            )
            
            # Get the ID of the newly inserted post
            post_id = cur.lastrowid
            
            conn.commit()
            print(f"✅ Post inserted successfully! Post ID: {post_id}")
            
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

# Test the function
add_post()
```

### 🔍 Verifying the Insertion

```python
print("\n🔍 VERIFYING THE INSERTION")
print("=" * 35)

def list_posts():
    """List all posts in the database"""
    try:
        with sqlite3.connect("blog.db") as conn:
            cur = conn.cursor()
            cur.execute("""
                SELECT p.PostID, p.Title, p.PublishedDate, u.Username
                FROM Posts p
                JOIN Users u ON p.AuthorID = u.UserID
                ORDER BY p.PublishedDate DESC
            """)
            
            posts = cur.fetchall()
            
            if not posts:
                print("No posts found.")
                return
                
            print("\n📝 BLOG POSTS")
            print("=" * 70)
            print(f"{'ID':<4} {'Title':<25} {'Date':<12} {'Author':<10}")
            print("-" * 70)
            
            for post in posts:
                title = post[1] if len(post[1]) <= 24 else post[1][:21] + "..."
                date = post[2][:10] if post[2] else "N/A"
                print(f"{post[0]:<4} {title:<25} {date:<12} {post[3]:<10}")
            
            print("=" * 70)
            
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")

# List posts to verify our insertion
list_posts()
```

---

## 4. 🛡️ Real-World Safeguards & Best Practices

### 🔗 Foreign Key Validation

```python
print("🔗 FOREIGN KEY VALIDATION")
print("=" * 35)

def validate_author(author_id):
    """Validate that an author exists in the database"""
    try:
        with sqlite3.connect("blog.db") as conn:
            conn.execute("PRAGMA foreign_keys = ON")
            cur = conn.cursor()
            
            cur.execute("SELECT Username FROM Users WHERE UserID = ?", (author_id,))
            result = cur.fetchone()
            
            if result:
                return True, result[0]  # Author exists, return username
            else:
                return False, None  # Author doesn't exist
                
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")
        return False, None

# Test validation
author_id = 1
exists, username = validate_author(author_id)
print(f"Author ID {author_id} exists: {exists}")
if exists:
    print(f"Author username: {username}")
```

### 💾 Transaction Management

```python
print("\n💾 TRANSACTION MANAGEMENT")
print("=" * 35)

def transactional_operation():
    """Example of proper transaction management"""
    try:
        with sqlite3.connect("blog.db") as conn:
            # Begin transaction implicitly with first execute
            conn.execute("PRAGMA foreign_keys = ON")
            cur = conn.cursor()
            
            # Multiple operations that should succeed or fail together
            cur.execute("INSERT INTO Posts (Title, Content, AuthorID) VALUES (?, ?, ?)",
                       ("Transaction Test", "Testing transaction handling", 1))
            
            # Simulate an error condition
            should_fail = False  # Change to True to test rollback
            if should_fail:
                raise Exception("Simulated error for transaction testing")
            
            cur.execute("UPDATE Posts SET Title = ? WHERE PostID = ?",
                       ("Updated Transaction Test", 1))
            
            # Commit only if all operations succeed
            conn.commit()
            print("✅ Transaction completed successfully")
            
    except Exception as e:
        print(f"❌ Transaction failed: {e}")
        # Rollback happens automatically due to context manager
        print("🔄 Changes rolled back due to error")

# Test transaction handling
transactional_operation()
```

### 🗑️ Soft Deletion Implementation

```python
print("\n🗑️ SOFT DELETION IMPLEMENTATION")
print("=" * 40)

def soft_delete_post(post_id):
    """Mark a post as inactive instead of deleting it"""
    try:
        with sqlite3.connect("blog.db") as conn:
            cur = conn.cursor()
            
            # Use soft deletion (update IsActive flag) instead of DELETE
            cur.execute("UPDATE Posts SET IsActive = 0 WHERE PostID = ?", (post_id,))
            
            conn.commit()
            print(f"✅ Post {post_id} marked as inactive (soft deletion)")
            
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")

def list_active_posts():
    """List only active posts"""
    try:
        with sqlite3.connect("blog.db") as conn:
            cur = conn.cursor()
            cur.execute("""
                SELECT PostID, Title, PublishedDate
                FROM Posts
                WHERE IsActive = 1
                ORDER BY PublishedDate DESC
            """)
            
            posts = cur.fetchall()
            
            print("\n📝 ACTIVE BLOG POSTS")
            print("=" * 50)
            for post in posts:
                print(f"{post[0]}: {post[1]} ({post[2][:10]})")
            
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")

# Test soft deletion
soft_delete_post(1)
list_active_posts()
```

---

## 5. 🧪 Hands-On Lab: Complete Blog Management System

### 🎯 Building a CLI Blog Manager

Let's create a complete command-line interface for managing our blog:

```python
print("🎯 COMPLETE BLOG MANAGEMENT SYSTEM")
print("=" * 50)

class BlogManager:
    """A complete blog management system"""
    
    def __init__(self, db_path="blog.db"):
        self.db_path = db_path
        self.setup_database()
    
    def setup_database(self):
        """Initialize database tables"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON")
                cur = conn.cursor()
                
                # Create tables if they don't exist
                cur.execute("""
                    CREATE TABLE IF NOT EXISTS Users (
                        UserID INTEGER PRIMARY KEY,
                        Username TEXT UNIQUE NOT NULL,
                        Email TEXT UNIQUE NOT NULL
                    )
                """)
                
                cur.execute("""
                    CREATE TABLE IF NOT EXISTS Posts (
                        PostID INTEGER PRIMARY KEY,
                        Title TEXT NOT NULL,
                        Content TEXT,
                        PublishedDate DATETIME DEFAULT CURRENT_TIMESTAMP,
                        AuthorID INTEGER,
                        IsActive INTEGER DEFAULT 1,
                        FOREIGN KEY (AuthorID) REFERENCES Users(UserID)
                    )
                """)
                
                # Insert sample users if needed
                sample_users = [
                    (1, 'alice_dev', 'alice@example.com'),
                    (2, 'bob_writer', 'bob@example.com'),
                    (3, 'charlie_author', 'charlie@example.com')
                ]
                
                cur.executemany("""
                    INSERT OR IGNORE INTO Users (UserID, Username, Email)
                    VALUES (?, ?, ?)
                """, sample_users)
                
                conn.commit()
                print("✅ Database setup complete")
                
        except sqlite3.Error as e:
            print(f"❌ Database setup error: {e}")
    
    def add_post(self):
        """Add a new blog post"""
        print("\n📝 ADD NEW BLOG POST")
        print("-" * 30)
        
        try:
            title = input("Title: ").strip()
            content = input("Content: ").strip()
            author_id = input("Author ID: ").strip()
            
            # Validation
            if not title or not content or not author_id.isdigit():
                print("❌ Invalid input. Please check your values.")
                return
                
            author_id = int(author_id)
            
            with sqlite3.connect(self.db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON")
                cur = conn.cursor()
                
                # Verify author exists
                cur.execute("SELECT Username FROM Users WHERE UserID = ?", (author_id,))
                author = cur.fetchone()
                
                if not author:
                    print(f"❌ Author with ID {author_id} does not exist.")
                    return
                
                # Insert post
                cur.execute("""
                    INSERT INTO Posts (Title, Content, AuthorID)
                    VALUES (?, ?, ?)
                """, (title, content, author_id))
                
                conn.commit()
                print(f"✅ Post added successfully by {author[0]}!")
                
        except sqlite3.Error as e:
            print(f"❌ Database error: {e}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def list_posts(self, active_only=True):
        """List blog posts"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cur = conn.cursor()
                
                if active_only:
                    query = """
                        SELECT p.PostID, p.Title, p.PublishedDate, u.Username
                        FROM Posts p
                        JOIN Users u ON p.AuthorID = u.UserID
                        WHERE p.IsActive = 1
                        ORDER BY p.PublishedDate DESC
                    """
                else:
                    query = """
                        SELECT p.PostID, p.Title, p.PublishedDate, u.Username, p.IsActive
                        FROM Posts p
                        JOIN Users u ON p.AuthorID = u.UserID
                        ORDER BY p.PublishedDate DESC
                    """
                
                cur.execute(query)
                posts = cur.fetchall()
                
                if not posts:
                    print("No posts found.")
                    return
                
                print(f"\n📝 {'ACTIVE ' if active_only else 'ALL '}BLOG POSTS")
                print("=" * 70)
                print(f"{'ID':<4} {'Title':<25} {'Date':<12} {'Author':<10} {'Status' if not active_only else ''}")
                print("-" * 70)
                
                for post in posts:
                    title = post[1] if len(post[1]) <= 24 else post[1][:21] + "..."
                    date = post[2][:10] if post[2] else "N/A"
                    status = "Active" if active_only or post[4] else "Inactive"
                    
                    if active_only:
                        print(f"{post[0]:<4} {title:<25} {date:<12} {post[3]:<10}")
                    else:
                        print(f"{post[0]:<4} {title:<25} {date:<12} {post[3]:<10} {status:<8}")
                
                print("=" * 70)
                
        except sqlite3.Error as e:
            print(f"❌ Database error: {e}")
    
    def update_post(self):
        """Update a blog post"""
        print("\n✏️ UPDATE BLOG POST")
        print("-" * 30)
        
        try:
            post_id = input("Post ID to update: ").strip()
            if not post_id.isdigit():
                print("❌ Invalid Post ID.")
                return
                
            post_id = int(post_id)
            
            # Check if post exists
            with sqlite3.connect(self.db_path) as conn:
                cur = conn.cursor()
                cur.execute("SELECT Title FROM Posts WHERE PostID = ?", (post_id,))
                post = cur.fetchone()
                
                if not post:
                    print(f"❌ Post with ID {post_id} does not exist.")
                    return
            
            print(f"Current title: {post[0]}")
            new_title = input("New title (press Enter to keep current): ").strip()
            new_content = input("New content (press Enter to keep current): ").strip()
            
            # Build update query dynamically based on what changed
            update_params = []
            update_values = []
            
            if new_title:
                update_params.append("Title = ?")
                update_values.append(new_title)
                
            if new_content:
                update_params.append("Content = ?")
                update_values.append(new_content)
                
            if not update_params:
                print("❌ No changes specified.")
                return
                
            update_values.append(post_id)
            update_query = f"UPDATE Posts SET {', '.join(update_params)} WHERE PostID = ?"
            
            with sqlite3.connect(self.db_path) as conn:
                cur = conn.cursor()
                cur.execute(update_query, update_values)
                conn.commit()
                
                print(f"✅ Post {post_id} updated successfully!")
                
        except sqlite3.Error as e:
            print(f"❌ Database error: {e}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def delete_post(self, soft_delete=True):
        """Delete a blog post (soft or hard delete)"""
        print("\n🗑️ DELETE BLOG POST")
        print("-" * 30)
        
        try:
            post_id = input("Post ID to delete: ").strip()
            if not post_id.isdigit():
                print("❌ Invalid Post ID.")
                return
                
            post_id = int(post_id)
            
            # Check if post exists
            with sqlite3.connect(self.db_path) as conn:
                cur = conn.cursor()
                cur.execute("SELECT Title FROM Posts WHERE PostID = ?", (post_id,))
                post = cur.fetchone()
                
                if not post:
                    print(f"❌ Post with ID {post_id} does not exist.")
                    return
            
            print(f"Post to delete: {post[0]}")
            confirm = input("Are you sure? (y/N): ").strip().lower()
            
            if confirm != 'y':
                print("❌ Deletion cancelled.")
                return
                
            with sqlite3.connect(self.db_path) as conn:
                cur = conn.cursor()
                
                if soft_delete:
                    # Soft delete (mark as inactive)
                    cur.execute("UPDATE Posts SET IsActive = 0 WHERE PostID = ?", (post_id,))
                    message = "marked as inactive"
                else:
                    # Hard delete (permanently remove)
                    cur.execute("DELETE FROM Posts WHERE PostID = ?", (post_id,))
                    message = "permanently deleted"
                
                conn.commit()
                print(f"✅ Post {post_id} {message}!")
                
        except sqlite3.Error as e:
            print(f"❌ Database error: {e}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def run(self):
        """Main CLI loop"""
        while True:
            print("\n📝 BLOG MANAGEMENT SYSTEM")
            print("=" * 30)
            print("1. Add new post")
            print("2. List active posts")
            print("3. List all posts (including inactive)")
            print("4. Update post")
            print("5. Soft delete post (mark inactive)")
            print("6. Hard delete post (permanent)")
            print("7. Exit")
            
            choice = input("\nEnter your choice (1-7): ").strip()
            
            if choice == '1':
                self.add_post()
            elif choice == '2':
                self.list_posts(active_only=True)
            elif choice == '3':
                self.list_posts(active_only=False)
            elif choice == '4':
                self.update_post()
            elif choice == '5':
                self.delete_post(soft_delete=True)
            elif choice == '6':
                self.delete_post(soft_delete=False)
            elif choice == '7':
                print("👋 Goodbye!")
                break
            else:
                print("❌ Invalid choice. Please try again.")

# Create and run the blog manager
blog_manager = BlogManager()
blog_manager.run()
```

---

## 6. 🚀 Bonus Exercises

### 🏆 Bonus Challenge 1: Bulk Post Insertion

```python
print("🏆 BONUS CHALLENGE 1: BULK POST INSERTION")
print("=" * 50)

def bulk_insert_posts():
    """Insert multiple posts at once using executemany"""
    try:
        # Sample posts data
        posts_data = [
            ("Python Data Analysis", "Pandas and NumPy are essential for data analysis...", 1),
            ("Web Development with Flask", "Flask is a lightweight web framework...", 2),
            ("Machine Learning Basics", "Introduction to scikit-learn and TensorFlow...", 1),
            ("Database Design Principles", "Normalization and relationship design...", 3)
        ]
        
        with sqlite3.connect("blog.db") as conn:
            conn.execute("PRAGMA foreign_keys = ON")
            cur = conn.cursor()
            
            # Use executemany for bulk insertion
            cur.executemany("""
                INSERT INTO Posts (Title, Content, AuthorID)
                VALUES (?, ?, ?)
            """, posts_data)
            
            conn.commit()
            print(f"✅ {len(posts_data)} posts inserted successfully!")
            
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")

# Run bulk insertion
bulk_insert_posts()
```

### 🏆 Bonus Challenge 2: Post Search Functionality

```python
print("\n🏆 BONUS CHALLENGE 2: POST SEARCH FUNCTIONALITY")
print("=" * 55)

def search_posts(search_term):
    """Search posts by title or content"""
    try:
        search_pattern = f"%{search_term}%"
        
        with sqlite3.connect("blog.db") as conn:
            cur = conn.cursor()
            
            cur.execute("""
                SELECT p.PostID, p.Title, p.PublishedDate, u.Username
                FROM Posts p
                JOIN Users u ON p.AuthorID = u.UserID
                WHERE (p.Title LIKE ? OR p.Content LIKE ?) AND p.IsActive = 1
                ORDER BY p.PublishedDate DESC
            """, (search_pattern, search_pattern))
            
            results = cur.fetchall()
            
            if not results:
                print(f"❌ No posts found matching '{search_term}'")
                return
                
            print(f"\n🔍 SEARCH RESULTS FOR '{search_term}':")
            print("=" * 60)
            for post in results:
                print(f"{post[0]}: {post[1]} ({post[2][:10]}) by {post[3]}")
                
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")

# Test search functionality
search_posts("Python")
```

### 🏆 Bonus Challenge 3: Author Statistics

```python
print("\n🏆 BONUS CHALLENGE 3: AUTHOR STATISTICS")
print("=" * 45)

def author_statistics():
    """Generate statistics about authors and their posts"""
    try:
        with sqlite3.connect("blog.db") as conn:
            cur = conn.cursor()
            
            cur.execute("""
                SELECT u.UserID, u.Username,
                       COUNT(p.PostID) as TotalPosts,
                       SUM(CASE WHEN p.IsActive = 1 THEN 1 ELSE 0 END) as ActivePosts,
                       MIN(p.PublishedDate) as FirstPost,
                       MAX(p.PublishedDate) as LatestPost
                FROM Users u
                LEFT JOIN Posts p ON u.UserID = p.AuthorID
                GROUP BY u.UserID, u.Username
                ORDER BY TotalPosts DESC
            """)
            
            stats = cur.fetchall()
            
            print("\n📊 AUTHOR STATISTICS")
            print("=" * 80)
            print(f"{'Author':<15} {'Total':<6} {'Active':<6} {'First Post':<12} {'Latest Post':<12}")
            print("-" * 80)
            
            for stat in stats:
                username = stat[1]
                total = stat[2] or 0
                active = stat[3] or 0
                first = stat[4][:10] if stat[4] else "N/A"
                latest = stat[5][:10] if stat[5] else "N/A"
                
                print(f"{username:<15} {total:<6} {active:<6} {first:<12} {latest:<12}")
                
            print("=" * 80)
            
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")

# Generate author statistics
author_statistics()
```

---

## 7. 📚 Comprehensive Summary

### 🎯 What We Learned Today:

1.  **DML Operations with Python:**
    - INSERT: Adding new records to databases
    - UPDATE: Modifying existing records
    - DELETE: Removing records (with caution!)

2.  **Dynamic Data Handling:**
    - Collecting user input and inserting into databases
    - Input validation and error handling
    - Parameterized queries for security

3.  **Best Practices:**
    - Foreign key validation and enforcement
    - Transaction management for data consistency
    - Soft deletion instead of permanent deletion
    - Proper error handling and user feedback

### 🔑 Key Concepts:

| Concept | Purpose | Example |
|:--------|:--------|:--------|
| **Parameterized Queries** | Prevent SQL injection | `cursor.execute("INSERT ... VALUES (?, ?)", (val1, val2))` |
| **Transaction Management** | Ensure data consistency | Use context managers (`with sqlite3.connect()`) |
| **Foreign Key Validation** | Maintain referential integrity | Check if referenced records exist before insertion |
| **Soft Deletion** | Preserve data history | Use `IsActive` flag instead of `DELETE` |

### 💡 Pro Tips:

1.  **Always use parameterized queries** to prevent SQL injection attacks
2.  **Validate user input** before database operations
3.  **Use transactions** for multiple related operations
4.  **Implement soft deletion** for business-critical data
5.  **Provide clear user feedback** for all operations

### ✅ Completion Checklist:

- [x] Understand DML operations (INSERT, UPDATE, DELETE) with Python
- [x] Practice parameterized queries for security
- [x] Implement input validation and error handling
- [x] Learn transaction management techniques
- [x] Build a complete blog management system
- [x] Complete bonus challenges

### 🔮 Next Steps:

1.  **Advanced SQL Operations**: JOINs across multiple tables
2.  **Database Optimization**: Indexes, query performance tuning
3.  **Web Integration**: Creating web APIs that interact with databases
4.  **ORM Frameworks**: SQLAlchemy, Django ORM for abstracted database access
5.  **Database Migration**: Managing schema changes over time

```python
print("\n🎉 Congratulations! You've mastered writing data to SQL databases using Python!")
print("   You can now build complete applications that interact with databases safely and efficiently!")
```

